In [1]:
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import gpflow
import tensorflow as tf
from gpflow.kernels import SquaredExponential as SE
from gpflow.kernels import RationalQuadratic as RQ
from gpflow.kernels import Linear, Periodic

In [4]:
import sys
from gpflow.ci_utils import ci_niter
sys.path.append('../code')
from gplar_pascal import GPLARmodel
from gpar.regression import GPARRegressor

In [ ]:
def generate_synthetic_data(kernels, N_train, N_test, noise, xmin=0, xmax=1):
    x = np.linspace(xmin, xmax, 1000).reshape(-1,1)
    idx = np.random.choice(1000,N_train+N_test,replace=False)
    idx_train, idx_test = idx[:N_train], idx[N_train:]
    inputx = x
    output, test, true = [],[],[]
    plt.figure(figsize=(20, len(kernels)*3))
    for i in range(len(kernels)):
        Kx = kernels[i](inputx)
        h = np.random.multivariate_normal(np.zeros(1000), Kx)[:,None]
        hh = h[idx_train]
        hh_test = h[idx_test]
        y = hh + np.random.normal(0,noise[i],N_train).reshape(-1,1)
        y_test = hh_test + np.random.normal(0,noise[i],N_test).reshape(-1,1)
        plt.subplot(len(kernels),1, i+1)
        plt.scatter(x[idx_train], y, c='black', label='Train', s=10)
        plt.scatter(x[idx_test], y_test, c='red', label='Test', s=10)
        plt.plot(x, h)
        plt.ylabel('output '+str(i))
        
        inputx = np.concatenate((inputx, h), axis=1)
        output.append(y)
        true.append(h)
        test.append(y_test)
    return x[idx_train], np.stack(output,axis=1)[:,:,0], \
            x, np.stack(true,axis=1)[:,:,0],\
            x[idx_test], np.stack(test, axis=1)[:,:,0]

In [ ]:
kernels_se = [SE(active_dims=[0],variance=1,lengthscales=0.05),
           SE(active_dims=[0],variance=1,lengthscales=0.02)
               +SE(active_dims=[1],variance=5,lengthscales=2.),
           SE(active_dims=[0],variance=1,lengthscales=0.05)
               +SE(active_dims=[1,2],variance=5,lengthscales=[3.,2.]),
           SE(active_dims=[0],variance=0.3,lengthscales=0.1)
               +SE(active_dims=[1,2,3],variance=1.,lengthscales=[2.,3.,3.])]


x_train, y_train, x_, h_, x_test, y_test = generate_synthetic_data(kernels_se, 100, 100, [0.1,0.1,0.1,0.1,0.1])

In [ ]:
def meanstd(y):
    mean, std = [],[]
    for i in range(y.shape[1]):
        available = ~np.isnan(y[:,i])
        y_i = y[available,i]
        mean.append(np.mean(y_i))
        std.append(np.std(y_i))
    return np.stack(mean), np.stack(std)

y_mean, y_std = meanstd(y_train)
y_train_norm = (y_train-mean)/std

In [ ]:
import pandas as pd
df = pd.read_csv('345_all.csv', dtype=np.float64)

In [ ]:
eeg_train_2 = df[['FZ','F1','F2','F3','F4','F5','F6']].copy()
eeg_train_2['FZ'].iloc[175:200] = None
eeg_train_2['F1'].iloc[175:200] = None
eeg_train_2['F2'].iloc[175:200] = None
eeg_train_2['F3'].iloc[50:75] = None
eeg_train_2['F4'].iloc[50:75] = None
eeg_train_2['F5'].iloc[100:125] = None
eeg_train_2['F6'].iloc[100:125] = None

eeg_test_2 = df[['time','F3','F4','F5','F6','FZ','F1','F2']].iloc[50:200].copy()
eeg_test_2['FZ'].iloc[:125] = None
eeg_test_2['F1'].iloc[:125] = None
eeg_test_2['F2'].iloc[:125] = None
eeg_test_2['F3'].iloc[25:] = None
eeg_test_2['F4'].iloc[25:] = None
eeg_test_2['F5'].iloc[:50] = None
eeg_test_2['F6'].iloc[:50] = None
eeg_test_2['F5'].iloc[75:] = None
eeg_test_2['F6'].iloc[75:] = None

In [ ]:
x_eeg = np.array(df['time']).reshape(-1,1)
y_eeg = np.array(eeg_train_2)
means_eeg, stds_eeg = [],[]
for i, name in enumerate(eeg_train_2.columns):
    available = ~np.isnan(y_eeg[:,i])
    y_i = y_eeg[available, i]
    means_eeg.append(np.mean(y_i))
    stds_eeg.append(np.std(y_i))
means_eeg, stds_eeg = np.stack(means_eeg), np.stack(stds_eeg)

def normalise_y(y_, means, stds): return (y_ - means)/stds
def unnormalise_y(y_, means, stds): return y_*stds + means

y_eeg_normalise = normalise_y(y_eeg ,means_eeg, stds_eeg)

In [ ]:
import pandas as pd
df = pd.read_csv('exchange_rates_2000-2010.csv', dtype=np.float64)
exchange = df[df.year<2008]
exchange = exchange[exchange.year>=2005]

In [ ]:
exchange_train_1 = exchange[[ 'USD/EUR', 'USD/GBP', 'USD/HKD', 'USD/KRW', 'USD/CAD', 'USD/JPY','USD/AUD']].copy()
exchange_train_1['USD/EUR'].iloc[:50]=None
exchange_train_1['USD/GBP'].iloc[:50]=None
exchange_train_1['USD/JPY'].iloc[500:550]=None
exchange_train_1['USD/AUD'].iloc[500:550]=None

exchange_test_1 = exchange[['year','USD/EUR', 'USD/GBP','USD/JPY','USD/AUD']].iloc[:550].copy()
exchange_test_1['USD/EUR'].iloc[50:] = None
exchange_test_1['USD/GBP'].iloc[50:] = None
exchange_test_1['USD/JPY'].iloc[:500] = None
exchange_test_1['USD/AUD'].iloc[:500] = None

In [ ]:
x_ex = np.array(exchange['year']).reshape(-1, 1)
y_ex = np.array(exchange_train_1)

In [ ]:
y_ex.shape

In [ ]:
means_exchange, stds_exchange = [],[]
for i in range(y_ex.shape[1]):
    available = ~np.isnan(y_ex[:,i])
    y_i = y_ex[available, i]
    means_exchange.append(np.mean(y_i))
    stds_exchange.append(np.std(y_i))

means_exchange, stds_exchange = np.stack(means_exchange), np.stack(stds_exchange)
def normalise_y(y_, means, stds): return (y_ - means)/stds
def unnormalise_y(y_, means, stds): return y_*stds + means
y_ex_normalise = normalise_y(y_ex, means_exchange, stds_exchange)

In [5]:
import numpy as np
import pandas as pd
df = pd.read_excel('data.xlsx', dtype=np.float64)

The measurements are made every 5 minutes and spans the entire month July of 2013
July 31 days, every day 24 hours, every hour 60 = 5*12 minitues
31*24*12 = 8928
if 1 means 1 day => total will be 31 days

In [6]:
df['time'] = np.linspace(0,31,8928)

In [7]:
df

,Bra height,Sot height,Cam height,Chi height,Bra speed,Sot speed,Cam speed,Chi speed,Bra temp,Sot temp,Cam temp,Chi temp,time
0,2.67,NaN,1.46,1.60,7.3,NaN,6.7,11.1,15.2,NaN,15.1,14.9,0.000000
1,2.69,NaN,1.49,1.63,7.2,NaN,6.9,10.8,15.2,NaN,15.1,14.9,0.003473
2,2.72,NaN,1.51,1.65,7.3,NaN,7.2,10.2,15.2,NaN,15.0,15.0,0.006945
3,2.74,NaN,1.54,1.67,7.2,NaN,7.4,10.7,15.1,NaN,15.0,15.0,0.010418
4,2.75,NaN,1.57,1.70,7.4,NaN,6.9,11.7,15.1,NaN,15.0,15.0,0.013890
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,2.63,2.29,2.13,2.24,4.7,8.2,2.4,8.4,18.1,18.5,18.7,18.6,30.986110
8924,2.61,2.26,2.09,2.22,4.5,8.6,2.2,8.2,18.3,18.5,18.6,18.5,30.989582
8925,2.57,2.21,2.05,2.19,4.3,8.3,2.0,7.8,18.2,18.5,18.6,18.4,30.993055
8926,2.55,2.19,2.01,2.16,4.8,6.6,2.8,7.7,18.2,18.5,18.6,18.4,30.996527


In [8]:
weather_train = df[['Cam temp','Chi temp', 'Bra height','Sot height','Cam height','Chi height','Bra speed','Sot speed','Cam speed','Chi speed','Bra temp','Sot temp']].copy()
weather_train['Cam temp'].iloc[3456:4608]=None #12-16 4days
weather_train['Chi temp'].iloc[4032:5184]=None #14-18 4days
weather_train['Bra temp'].iloc[6912:8064]=None #24-28 4days
weather_train['Sot temp'].iloc[7488:8640]=None #26-30 4days

weather_test = df[['time','Cam temp','Chi temp', 'Bra temp','Sot temp']].iloc[3456:8640].copy()
weather_test['Cam temp'].iloc[1152:] = None
weather_test['Chi temp'].iloc[:576] = None
weather_test['Chi temp'].iloc[1728:] = None
weather_test['Bra temp'].iloc[:3456] = None
weather_test['Bra temp'].iloc[4608:] = None
weather_test['Sot temp'].iloc[:4032] = None

In [9]:
x_wt = np.array(df['time']).reshape(-1, 1)
y_wt = np.array(weather_train)
means_weather, stds_weather = [],[]
for i in range(y_wt.shape[1]):
    available = ~np.isnan(y_wt[:,i])
    y_i = y_wt[available, i]
    means_weather.append(np.mean(y_i))
    stds_weather.append(np.std(y_i))

means_weather, stds_weather = np.stack(means_weather), np.stack(stds_weather)
def normalise_y(y_, means, stds): return (y_ - means)/stds
def unnormalise_y(y_, means, stds): return y_*stds + means
y_wt_normalise = normalise_y(y_wt, means_weather, stds_weather)

In [ ]:
gpar = GPARRegressor(scale=0.02,
                    linear=False, nonlinear=True, nonlinear_scale=1.0,
                    noise=0.01,
                    impute=True, replace=True, normalise_y=False)
gpar_back = GPARRegressor(scale=0.02,
                    linear=False, nonlinear=True, nonlinear_scale=1.0,
                    noise=0.01,
                    impute=True, replace=True, normalise_y=False)

In [ ]:
x_ind = np.linspace(x_wt.min(), x_wt.max(), 300)
weather_gpar = GPARRegressor(scale=0.2,
                          linear=True, linear_scale=10.,
                          nonlinear=True, nonlinear_scale=1.,
                          noise=0.1,
                          impute=True, replace=True, normalise_y=True,
                          x_ind=x_ind)
weather_gpar_back = GPARRegressor(scale=0.2,
                          linear=True, linear_scale=10.,
                          nonlinear=True, nonlinear_scale=1.,
                          noise=0.1,
                          impute=True, replace=True, normalise_y=True,
                          x_ind=x_ind)

In [ ]:
weather_gpar_back = GPARRegressor(scale=0.2,
                          linear=True, linear_scale=10.,
                          nonlinear=True, nonlinear_scale=1.,
                          noise=0.1,
                          impute=True, replace=True, normalise_y=True,
                          x_ind=x_ind)

In [ ]:
weather_gpar.fit(x_wt, y_wt)

In [ ]:
y_wt_back = y_wt[:,::-1].copy()
weather_gpar_back.fit(x_wt, y_wt_back)

In [ ]:
backward_samples = weather_gpar_back.sample(x_wt[3456:5000,0], num_samples=50, latent=True, posterior=True)

In [ ]:
backward_samples_2 = weather_gpar_back.sample(x_wt[5000:7000,0], num_samples=50, latent=True, posterior=True)

In [ ]:
backward_samples_3 = weather_gpar_back.sample(x_wt[7000:8640,0], num_samples=50, latent=True, posterior=True)

In [ ]:
backward_samples_ = np.concatenate([backward_samples, backward_samples_2,backward_samples_3], axis=1)

In [ ]:
backward_samples_.shape

In [ ]:
z1 = np.linspace(np.min(x_wt),np.max(x_wt),300).reshape(300,1)
z2 = np.linspace(np.min(x_wt),np.max(x_wt),300).reshape(300,1)

In [ ]:
import torch
from lab.torch import B

In [ ]:
z = np.linspace(np.min(x_wt),np.max(x_wt),300).reshape(300,1)
samples = weather_gpar.sample(z, num_samples=100, latent=True, posterior=True)
means, std = np.mean(samples,axis=0), np.std(samples,axis=0)

In [ ]:
z = np.linspace(np.min(x_wt),np.max(x_wt),300).reshape(300,1)
samples = weather_gpar_back.sample(z, num_samples=100, latent=True, posterior=True)
means, std = np.mean(samples,axis=0), np.std(samples,axis=0)

In [ ]:
with open('backwards_gpar.npy', 'wb') as f:
    np.save(f, means)
    np.save(f, std)

In [10]:
with open('forward_gpar.npy', 'rb') as f:
    a = np.load(f)
    b = np.load(f)

In [ ]:
with open('backwards_gpar.npy', 'rb') as f:
    c = np.load(f)
    d = np.load(f)

In [ ]:
c = unnormalise_y(c, means_weather[::-1], stds_weather[::-1])
d = unnormalise_y(d, 0, stds_weather[::-1])

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3456:8640,0]
mean, lower, upper = np.mean(forward_samples_, axis=0), np.percentile(forward_samples_, 2.5, axis=0), np.percentile(forward_samples_, 100-2.5, axis=0)
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)

    
    plt.plot(x, mean[:,p], c='red', label = 'forward')
    plt.fill_between(x, lower[:,p], upper[:,p], facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.ylabel(name)
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3456:8640,0]
mean, lower, upper = np.mean(backward_samples_, axis=0), np.percentile(backward_samples_, 2.5, axis=0), np.percentile(backward_samples_, 100-2.5, axis=0)
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)

    
    plt.plot(x, mean[:,11-p], c='red', label = 'forward')
    plt.fill_between(x, lower[:,11-p], upper[:,11-p], facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.ylabel(name)
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3456:8640,0]
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)

    
    mean = np.mean((Hmeans[p]+BHmeans[p])/2.0,axis=0)
    var = np.mean(((Hmeans[p]+BHmeans[p])/2.0)**2 + (Hvars[p]+BHvars[p])/4.0 + gplar_weather.likelihoods[p].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (days)')
    plt.ylabel(name)
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3456:8640,0]
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)

    
    mean = np.mean(Hmeans[p],axis=0)
    var = np.mean(Hmeans[p]**2 + Hvars[p],axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    meanb = np.mean(BHmeans[p],axis=0)
    varb = np.mean(BHmeans[p]**2 + BHvars[p],axis=0) - meanb**2
    meanb, varb = meanb[:,0],varb[:,0]
    meanb, stdb = meanb* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(varb)
    
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.plot(x, meanb, c='green', label = 'GPLAR')
    plt.fill_between(x, meanb-1.96*stdb, meanb+1.96*stdb, facecolor='tab:green', alpha=.25)
    
    plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (days)')
    plt.ylabel(name)
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3456:8640,0]
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)

    
    mean = np.mean(Hmeans[p],axis=0)
    var = np.mean(Hmeans[p]**2 + Hvars[p],axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    #meanb = np.mean(BHmeans[p],axis=0)
    #varb = np.mean(BHmeans[p]**2 + BHvars[p],axis=0) - meanb**2
    #meanb, varb = meanb[:,0],varb[:,0]
    #meanb, stdb = meanb* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(varb)
    
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    #plt.plot(x, meanb, c='green', label = 'GPLAR')
    #plt.fill_between(x, meanb-1.96*stdb, meanb+1.96*stdb, facecolor='tab:green', alpha=.25)
    
    plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (days)')
    plt.ylabel(name)
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3456:8640,0]
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)

    
    mean = np.mean(Hmeans[p],axis=0)
    var = np.mean(Hmeans[p]**2 + Hvars[p],axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    #meanb = np.mean(BHmeans[p],axis=0)
    #varb = np.mean(BHmeans[p]**2 + BHvars[p],axis=0) - meanb**2
    #meanb, varb = meanb[:,0],varb[:,0]
    #meanb, stdb = meanb* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(varb)
    
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    #plt.plot(x, meanb, c='green', label = 'GPLAR')
    #plt.fill_between(x, meanb-1.96*stdb, meanb+1.96*stdb, facecolor='tab:green', alpha=.25)
    
    plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (days)')
    plt.ylabel(name)
    plt.ylim(y_wt[~np.isnan(y_wt[:,p]), p].min()-5, y_wt[~np.isnan(y_wt[:,p]), p].max()+5)
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, 5))
x = x_wt[3456:8640,0]
name = 'Bra temp'
p = list(weather_train.columns).index(name)

for i in range(len(Hmeans[p])):
    mean = np.mean(Hmeans[p][i],axis=0)
    var = np.mean(Hmeans[p][i]**2 + Hvars[p][i],axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    if i==0: 
        plt.plot(x, mean, color='red', label='temproal')
        plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    else:
        plt.plot(x, mean)
        plt.fill_between(x, mean-1.96*std, mean+1.96*std, alpha=.25)
    
 
plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, 5))
x = x_wt[3456:8640,0]
name = 'Bra temp'
p = list(weather_train.columns).index(name)

for i in range(len(Hmeans[p])):
    mean = np.mean(Hmeans[p][i],axis=0)
    var = np.mean(Hmeans[p][i]**2 + Hvars[p][i],axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    if i==0: 
        plt.plot(x, mean, color='red', label='temproal')
        plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    else:
        plt.plot(x, mean)
        plt.fill_between(x, mean-1.96*std, mean+1.96*std, alpha=.25)
    
 
plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
plt.legend()

In [11]:
M=300
x_ind = np.linspace(x_wt.min(), x_wt.max(), M).reshape(-1,1)

gplar_weather = GPLARmodel(x_wt, y_wt_normalise, M, 100, 1, x_ind, a, b, 
                    minibatch_size=1000,
                    white=False,
                    scale=0.2, per=True,
                    linear=True, linear_scale=10.,
                    nonlinear=True, nonlinear_scale=1.0,
                    nonlinear_dependent=True,
                    noise_inner=1e-5, noise_obs=1e-3,
                    num_samples=1)

initializing temporal layers


In [12]:
gplar_weather.boosting_initialization()

In [14]:
for layers in gplar_weather.boosting_backwards_layers[1:]:
    for layer in layers:
        layer.q_sqrt = gpflow.base.Parameter(layer.q_sqrt*1e-3, transform=gpflow.utilities.triangular())

In [ ]:
for layer in gplar_weather.forwards_layers:
    gpflow.set_trainable(layer.q_mu,False)
    gpflow.set_trainable(layer.q_sqrt,False)
for layer in gplar_weather.backwards_layers:
    gpflow.set_trainable(layer.q_mu,False)
    gpflow.set_trainable(layer.q_sqrt,False)

In [ ]:
gpflow.set_trainable(gplar_weather.inducing_locations, False)

In [ ]:
gplar.boosting_initialization(y_wt_normalise,100)

In [ ]:
gplar.boosting_backwards_initialization(y_wt_normalise,100)

In [ ]:
for layers in gplar.boosting_layers[1:]:
    for layer in layers:
        layer.q_sqrt = gpflow.base.Parameter(layer.q_sqrt.value() * 1e-5, 
                                 transform=gpflow.utilities.triangular())

In [15]:
minibatch_size = 1000
train_ds = tf.data.Dataset.from_tensor_slices((x_wt, y_wt_normalise)).repeat()

In [ ]:
gplar_weather.num_data

In [41]:
data = next(train_it)
gplar_weather.maximum_log_likelihood_objective(*data)

<tf.Tensor: shape=(), dtype=float64, numpy=19911.515687312352>

In [43]:
@tf.function(autograph=False)
def optimization_step(optimizer, model, data):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.trainable_variables)
        objective = -model.maximum_log_likelihood_objective(*data)
        grads = tape.gradient(objective, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return objective

def run_adam(model, iterations):
    learning_rate = 0.0001
    adam = tf.optimizers.Adam(learning_rate=learning_rate)
    train_it = iter(train_ds.batch(minibatch_size))
    for step in range(iterations):
        neg_elbo= optimization_step(adam, model, next(train_it))
        elbo = -neg_elbo
        if step%100 == 0:
            print(elbo.numpy())

maxiter = ci_niter(50000)
run_adam(gplar_weather,  maxiter)

-14033762.65988556
-11391172.258874895
-7303718.278227894
-8776952.911653325
-7358457.206706711
-3274676.349842146


KeyboardInterrupt: 

In [ ]:
def propagate(self, X, full_cov=False, S=1, zs=None):
        
    sX = tf.tile(tf.expand_dims(X, 0), [S, 1, 1]) # [S,N,1]
    Hmeans, Hvars = [], []
    zs = zs or [None, ] * len(self.temporal_layers) # [None, None, ..., None]
    for temporal_layer, z, i in zip(self.temporal_layers, zs, range(self.num_outputs)):
        Ht, Hmeant, Hvart = temporal_layer.sample_from_conditional(sX, z=z, full_cov=full_cov)
        Hmeans.append([Hmeant,])
        Hvars.append([Hvart,])
        
        for layers in self.boosting_layers[i+1:]:
            Hf, Hmeanf, Hvarf = layers[i].sample_from_conditional(Ht, z=z, full_cov=full_cov)
            Hmeans[i].append(Hmeanf)
            Hvars[i].append(Hvarf)
        for layers in self.boosting_backwards_layers[self.num_outputs-i:]:
            Hb, Hmeanb, Hvarb = layers[self.num_outputs-i-1].sample_from_conditional(Ht, z=z, full_cov=full_cov)
            Hmeans[i].append(Hmeanb)
            Hvars[i].append(Hvarb)

    return Hmeans, Hvars

In [ ]:
Hmeans, Hvars = propagate(gplar_weather,x_wt[3456:8640],S=50)

In [ ]:
len(Hmeans[0])

In [ ]:
@tf.function(autograph=False)
def optimization_step(optimizer, model, data):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.trainable_variables)
        objective = -model.maximum_log_likelihood_objective(*data)
        grads = tape.gradient(objective, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return objective

def run_adam(model, iterations):
    adam = tf.optimizers.Adam(0.0001)
    train_it = iter(train_ds.batch(minibatch_size))
    for step in range(iterations):
        neg_elbo= optimization_step(adam, model, next(train_it))
        elbo = -neg_elbo
        if step%500 == 0:
            print(elbo.numpy())

maxiter = ci_niter(50000)
run_adam(gplar_weather,  maxiter)

In [ ]:
gpflow.utilities.print_summary(gplar_weather, fmt='notebook')

In [ ]:
Hmean, Hvar = gplar.propagate(x_wt[3456:5184,:],S=100)

In [ ]:
Hmeans, Hvars = gplar_weather.propagate(x_wt[3456:8640,:],S=50)

In [ ]:
Hmean, Hvar = gplar.propagate(x_ex,S=100)

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3168:5472,0]
for i, name in enumerate(['Bra speed','Sot speed','Cam speed']):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)
    mean = np.mean(Hmean[p],axis=0)
    var = np.mean(Hmean[p]**2 + Hvar[p] + gplar.likelihoods[p].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3168:5472, p], c='black', s=15, marker='+')
    #plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.ylabel(name)
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3456:5184,0]
for i, name in enumerate(['Bra height','Sot height','Cam height']):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)
    mean = np.mean(Hmean[p],axis=0)
    var = np.mean(Hmean[p]**2 + Hvar[p] + gplar.likelihoods[p].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3456:5184, p], c='black', s=15, marker='+')
    #plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.ylabel(name)
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3168:5472,0]
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)
    mean = np.mean((Hmeans[p]+BHmeans[p])/2.0,axis=0)
    var = np.mean(((Hmeans[p]+BHmeans[p])/2.0)**2 + (Hvars[p]+BHvars[p])/4.0 + gplar.likelihoods[p].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3168:5472, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.ylabel(name)
    plt.legend()
    if name == 'Cam temp': plt.xlim(11.,17.)
    if name == 'Chi temp': plt.xlim(13.,19.)

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3456:8640,0]
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)
    mean = np.mean((Hmeans[p]+BHmeans[p])/2.0,axis=0)
    var = np.mean(((Hmeans[p]+BHmeans[p])/2.0)**2 + (Hvars[p]+BHvars[p])/4.0 + gplar.likelihoods[p].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3456:8640, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (days)')
    plt.ylabel(name)
    plt.legend()
    if name == 'Cam temp': plt.xlim(11.,17.)
    if name == 'Chi temp': plt.xlim(13.,19.)
    if name == 'Bra temp': plt.xlim(23.,29.)
    if name == 'Sot temp': plt.xlim(25.,31.)

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3168:5472,0]
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)
    mean = np.mean(Hmean[p],axis=0)
    var = np.mean(Hmean[p]**2 + Hvar[p] + gplar.likelihoods[p].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3168:5472, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.ylabel(name)
    plt.legend()
    if name == 'Cam temp': plt.xlim(11.,17.)
    if name == 'Chi temp': plt.xlim(13.,19.)

In [ ]:
fig = plt.figure(figsize=(20, y_wt.shape[1]*5))
x = x_wt[3168:5472,0]
for i, name in enumerate(weather_test.columns[1:]):
    p = list(weather_train.columns).index(name)
    plt.subplot(y_wt.shape[1],1,i+1)
    mean = np.mean(Hmean[p],axis=0)
    var = np.mean(Hmean[p]**2 + Hvar[p] + gplar.likelihoods[p].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_weather[p] + means_weather[p], stds_weather[p] * np.sqrt(var)
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_wt[3168:5472, p], c='black', s=15, marker='+')
    plt.scatter(weather_test['time'], weather_test[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.ylabel(name)
    plt.legend()
    if name == 'Cam temp': plt.xlim(11.,17.)
    if name == 'Chi temp': plt.xlim(13.,19.)

In [ ]:
def predict_log_density(Hmean, Hvar, train, test, gplar):
    log_density = []
    for i, name in enumerate(test.columns[1:]):
        Ynew = np.array(test[name])
        p = list(train.columns).index(name)
        mean = Hmean[p].numpy()
        variance = Hvar[p].numpy()
        mu = mean[:,~np.isnan(Ynew),0] * stds_weather[p] + means_weather[p] #[S,N]
        var = variance[:,~np.isnan(Ynew),0] * stds_weather[p]**2 + gplar.likelihoods[p].variance #[S,N]
        l = -0.5*(np.log(2*np.pi)+np.log(var)+np.square(Ynew[~np.isnan(Ynew)] - mu)/var) #[S,N]
        log_num_samples = tf.math.log(tf.cast(mean.shape[0], l.dtype))
        log_likelihood = tf.reduce_sum(tf.reduce_logsumexp(l - log_num_samples, axis=0)) #[N,] -> scalar
        log_density.append([name, log_likelihood.numpy()])
    return log_density

def gplar_smse(Hmean, Hvar, train, test):
    smse = []
    for i, name in enumerate(test.columns[1:]):
        Ynew = np.array(test[name])
        p = list(train.columns).index(name)
        mean = Hmean[p].numpy()
        mu = mean[:,~np.isnan(Ynew),0] * stds_weather[p] + means_weather[p]
        smse_i = np.mean((mu-Ynew[~np.isnan(Ynew)])**2)/np.mean((np.mean(Ynew[~np.isnan(Ynew)])-Ynew[~np.isnan(Ynew)])**2)
        smse.append([name, smse_i])
    return smse

In [ ]:
def predict_log_density_gpar(train, test, samples, reverse=False):
    means, vars = np.mean(samples,axis=0), np.var(samples,axis=0)
    log_density = []
    for i, name in enumerate(test.columns[1:]):
        y = np.array(test[name])
        if reverse: p = means.shape[1]-list(train.columns).index(name)-1
        else: p = list(train.columns).index(name)
        Y = y[~np.isnan(y)]
        mu, var = means[~np.isnan(y),p], vars[~np.isnan(y),p]
        log_likelihood = -0.5*(np.log(2*np.pi)+np.log(var)+np.square(Y - mu)/var)
        log_density.append([name,np.sum(log_likelihood)])
    return log_density

def gpar_smse(train, test, samples, reverse=False):
    smse=[]
    means = np.mean(samples, axis=0)
    for i, name in enumerate(test.columns[1:]):
        y = np.array(test[name])
        if reverse: p = means.shape[1]-list(train.columns).index(name)-1
        else: p = list(train.columns).index(name)
        mu = means[~np.isnan(y),p]
        smse_i = np.mean((mu-y[~np.isnan(y)])**2)/np.mean((np.mean(y[~np.isnan(y)])-y[~np.isnan(y)])**2)
        smse.append([name, smse_i])
    return smse

Cam temp and Chi temp are the last two output

In [ ]:
predict_log_density(Hmean, Hvar, weather_train, weather_test)

In [ ]:
gplar_smse(Hmean, Hvar,  weather_train, weather_test)

Cam temp and Chi temp are the first two output

In [ ]:
predict_log_density(Hmean, Hvar, weather_train, weather_test)

In [ ]:
gplar_smse(Hmean, Hvar,  weather_train, weather_test)

## Bi-directional GPLAR

In [ ]:
predict_log_density(BHmeans, BHvars, weather_train, weather_test, gplar_weather)

In [ ]:
gplar_smse(BHmeans, BHvars,  weather_train, weather_test)

In [ ]:
predict_log_density_gpar(weather_train, weather_test, backward_samples_, reverse=True)

In [ ]:
gpar_smse(weather_train, weather_test, backward_samples_, reverse=True)

In [ ]:
gplar_smse(Hmeans, Hvars,  weather_train, weather_test)

In [ ]:
predict_log_density(Hmeans, Hvars, weather_train, weather_test, gplar_weather)

In [ ]:
predict_log_density_gpar(weather_train, weather_test, forward_samples_)

In [ ]:
gpar_smse(weather_train, weather_test, forward_samples_)

In [ ]:
fig = plt.figure(figsize=(20, y_ex.shape[1]*5))
x = x_ex[:,0]
for i, name in enumerate(exchange_train_1.columns):
    plt.subplot(y_ex.shape[1],1,i+1)
    mean = np.mean(Hmean[i],axis=0)
    var = np.mean(Hmean[i]**2 + Hvar[i] + gplar.likelihoods[i].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_exchange[i] + means_exchange[i], stds_exchange[i] * np.sqrt(var)
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.fill_between(x, mean-1.96*std, mean+1.96*std, facecolor='tab:red', alpha=.25)
    
    plt.scatter(x, y_ex[:, i], c='black', s=15, marker='+')
    if name in exchange_test_1.columns:
        plt.scatter(exchange_test_1['year'], exchange_test_1[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.ylabel(f'{name} (volt)')
    plt.legend()

In [ ]:
fig = plt.figure(figsize=(20, y_eeg.shape[1]*5))
x = x_eeg[:,0]
for i, name in enumerate(eeg_train_2.columns):
    plt.subplot(y_eeg.shape[1],1,i+1)
    
    mean = np.mean((Hmeans[i] + BHmeans[i])/2.,axis=0)
    meanf = np.mean(Hmeans[i], axis=0)
    varf = np.mean(Hmeans[i]**2+Hvars[i], axis=0)-meanf**2
    meanb = np.mean(BHmeans[i],axis=0)
    varb = np.mean(BHmeans[i]**2+BHvars[i], axis=0)-meanb**2
    var = np.mean(((Hmeans[i]+BHmeans[i])/2.0)**2 + (Hvars[i]+BHvars[i])/4.0 + gplar.likelihoods[i].variance,axis=0) - mean**2
    mean, var = mean[:,0],var[:,0]
    mean, std = mean* stds_eeg[i] + means_eeg[i], stds_eeg[i] * np.sqrt(var)
    meanf, meanb = meanf[:,0]*stds_eeg[i]+means_eeg[i], meanb[:,0]*stds_eeg[i]+means_eeg[i]
    stdf, stdb = stds_eeg[i]*np.sqrt(varf[:,0]), stds_eeg[i]*np.sqrt(varb[:,0])
    
    plt.plot(x, mean, c='red', label = 'GPLAR')
    plt.plot(x, meanf, c='blue', label='forward')
    plt.fill_between(x, meanf-1.96*stdf, meanf+1.96*stdf, facecolor='tab:blue', alpha=.25)
    plt.plot(x, meanb, c='green', label='backward')
    plt.fill_between(x, meanb-1.96*stdb, meanb+1.96*stdb, facecolor='tab:green', alpha=.25)
    
    plt.scatter(x, y_eeg[:, i], c='black', s=15, marker='+')
    if name in eeg_test_2.columns:
        plt.scatter(eeg_test_2['time'], eeg_test_2[name], c='green', s=20)
    plt.xlabel('Time (second)')
    plt.xlim(0, 1)
    plt.ylabel(f'{name} (volt)')
    plt.legend()

In [ ]:
f = open('pascal-weather-NL-L-path2-reverse.txt','w+')
f.write('nonlinear:'+'\n') 
for layers in gplar.boosting_layers[1:]:
    current = ''
    for layer in layers:
        current += str(layer.kernel.kernels[0].variance.value().numpy()) + ' '
    f.write(current + '\n')
f.write('back:'+'\n') 
for layers in gplar.boosting_backwards_layers[1:]:
    current = ''
    for layer in layers:
        current += str(layer.kernel.kernels[0].variance.value().numpy()) + ' '
    f.write(current + '\n')
f.write('linear:'+'\n') 
for layers in gplar.boosting_layers[1:]:
    current = ''
    for layer in layers:
        current += str(layer.kernel.kernels[1].variance.value().numpy()) + ' '
    f.write(current + '\n')
f.write('back:'+'\n') 
for layers in gplar.boosting_backwards_layers[1:]:
    current = ''
    for layer in layers:
        current += str(layer.kernel.kernels[1].variance.value().numpy()) + ' '
    f.write(current + '\n')
f.write('temporal:'+'\n') 
for layer in gplar.temporal_layers:
    f.write(str(layer.kernel.kernels[1].variance.value().numpy())+'\n')